# Monasca Bootcamp
## Hands on Lab

https://github.com/witekest/monasca-bootcamp

---
### [Roland Hochmuth](https://www.linkedin.com/in/rolandhochmuth)

company: Hewlett Packard Enterprise

email: <roland.hochmuth@hpe.com>

### Witek Bedyk

company: Fujitsu EST

email: <witold.bedyk@est.fujitsu.com>

# The Monasca team attending the Austin Summit

* Stefano Canepa (Hewlett Packard Enterprise)

* James Gu (SUSE)

* Dan ? (SUSE)

* Haruki Yamanashi (Fujitsu Limited)

* Cristiano Bellucci (Fujitsu EST)

* and more

# Running the iPython notebook
---
This is an iPython/Jupyter notebook that you can run on your own. You will require the following installed on your system:

* iPython

    * See, https://ipython.org/install.html

* An OpenStack DevStack install with the Monasca DevStack plugin installed.

    * See, https://github.com/openstack/monasca-log-api/tree/master/devstack

# Running notebook on your worshop instance
---
* Set up the SSH tunnel to your instance
    
    * `ssh -NfL localhost:8888:localhost:8889 ubuntu@<your_instance_ip>`
    
* SSH to your instance and start the notebook

    * `ssh ubuntu@<your_instance_ip>`
    * `/opt/jupyter/bin/jupyter notebook --no-browser --port=8889 --notebook-dir monasca-bootcamp/`
    
* Copy the URL and open it in your local browser. Remember to set the port number to `8888`.

* Open `Monasca Bootcamp.ipynb` notebook.

# Agenda
---
* Architecture and Overview
* API, CLI (python-monascaclient) and client hands-on
* Agent hands-on
* Horizon, Grafana, Kibana hands-on

# Architecture
---
![Monasca Architecture](architecture.png)

# Logging
---
![Logging Architecture](Logging_Architecture.png)

# Start the services

In [1]:
!~/scripts/start_services.sh
!~/scripts/list_services.sh

Starting rabbitmq-server
Starting apache2
Starting zookeeper
Starting storm-nimbus
Starting storm-supervisor
Starting mysql
Starting kafka
Starting influxdb
Starting elasticsearch
Starting memcached
Starting monasca-thresh
Starting monasca-notification
Starting monasca-persister
Starting monasca-log-transformer
Starting monasca-log-persister
Starting monasca-log-metrics
Starting monasca-api
Starting monasca-log-api
Starting monasca-agent
Starting monasca-log-agent
Starting grafana-server
Starting kibana
Service rabbitmq-server is active.
Service apache2 is active.
Service zookeeper is active.
Service storm-nimbus is active.
Service storm-supervisor is active.
Service mysql is active.
Service kafka is active.
Service influxdb is active.
Service elasticsearch is active.
Service memcached is active.
Service monasca-thresh is active.
Service monasca-notification is active.
Service monasca-persister is active.
Service monasca-log-transformer is active.
Service monasca-log-persister is activ

# Import libraries and initialization
---
Let's first import some libraries used in the rest of the notebook.

In [2]:
import datetime
import json
import time

# Import the Monasca and Keystone clients
from monascaclient import client
from monascaclient import ksclient

# Initialize the Keystone and Monasca Client
---
We'll be connecting to the DevStack VM and using the `mini-mon` project and username.

In [4]:
KEYSTONE_URL = 'http://<your_instance_ip>:35357/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [5]:
# Authenticate to Keystone
keystone_client = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD,
                                   project_name=PROJECT_NAME)

# Create the Monasca client
monasca_client = client.Client('2_0', keystone_client.monasca_url, token=keystone_client.token)

# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME

env: OS_PROJECT_NAME=mini-mon
env: OS_PASSWORD=password
env: OS_AUTH_URL=http://192.168.10.6:35357/v3
env: OS_USERNAME=mini-mon


# Using the API
---

The [Monasca API](https://github.com/openstack/monasca-api/blob/master/docs/monasca-api-spec.md) has the following resources:

* Versions

* Metrics

* Metrics Measurements

* Metrics Statistics

* Metrics Names

* Notification Methods

* Alarm Definitions

* Alarms

* Alarms Count

* Alarms State History

Please consult the spec for the full details.

# Metrics
---
* GET, POST /v2.0/metrics

* name (string(255), required) - The name of the metric. Naming conventions for metric names:

    * lowercase, `.` to delimit groups, `_` (snake case) to delimit words, with a unit of measurement often as the suffix.
    
    * e.g. cpu.user_perc, kafka.consumer_lag

* dimensions ({string(255): string(255)}, optional) - A dictionary consisting of (key, value) pairs used to uniquely identify a metric and slice and dice on.

* timestamp (string, required) - The timestamp in milliseconds from the Epoch.

* value (float, required) - Value of the metric.

* value_meta ({string(255): string(2048)}, optional) - A dictionary consisting of (key, value) pairs used to describe the metric.

    * Examples: status_code, msg

* tenant_id: Tenant ID to create metrics on behalf of.

    * This parameter can be used to submit metrics from one tenant, to another.
    * Requires the delegate role.

# Dimensions
---

* A dictionary of (key, value) pairs that are used to uniquely identify a metric.

* Used to slice and dice metrics when querying.

* Examples: hostname, service, component, region zone, resource_id, ...

* Dimensions can be anything you want, but naming conventions should be adopted for consistency.

* Examples of dimension keys are the following: hostname, region, zone, service, component, process, ...

# Example: Metrics request body
---
In this example, we are reporting the status of a HTTP check as a binary value, 0 or 1, for a specific host. The API is down, and the status code and msg returned are returned as meta data.
```
{
	name: http_status,
	dimensions: {
		url: http://service.domain.com:80,
		region: uswest,
		zone: 1,
		service: compute
	}
	timestamp: 1461600900000, /* milliseconds */
	value: 1.0,
	value_meta: {
		status_code: 500,
		msg: Internal server error
	}
}
```

# Help: Create metrics
---

In [6]:
!monasca help metric-create

usage: monasca metric-create [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--value-meta <KEY1=VALUE1,KEY2=VALUE2...>]
                             [--time <UNIX_TIMESTAMP>]
                             [--project-id <CROSS_PROJECT_ID>]
                             <METRIC_NAME> <METRIC_VALUE>

Create metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to create.
  <METRIC_VALUE>        Metric value.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to create a metric dimension. This
                        can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --value-meta <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair for extra information about a value.
       

# Example: Create a metric
Let's create a metric with the metric `openstack.handson_status` and some dimensions 

In [7]:
!monasca metric-create --dimensions region=useast,state=MA,city=Boston,session=monasca openstack.handson_status 1.0

Successfully created metric


# List metrics
Lists all the unique metrics in the system

* GET /v2.0/metrics

* A unique metric is identitifed by it's name and dimensions.

* Starttime, endtime, offset and limit paramaters are available.

* Note, if you are monitoring OpenStack resources, such as VMs, based on the amount of churn (VMs being created/destroyed) in the system and your retention policy, it is recommended to specify the starttime, to limit the amount of metrics returned, such that queries return in a reasonable amount of time.

# Help: List Metrics
---


In [8]:
!monasca help metric-list

usage: monasca metric-list [--name <METRIC_NAME>]
                           [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--starttime <UTC_START_TIME>]
                           [--endtime <UTC_END_TIME>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>] [--tenant-id <TENANT_ID>]

List metrics for this tenant.

Optional arguments:
  --name <METRIC_NAME>  Name of the metric to list.
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00

# Example: List metrics

In [9]:
!monasca metric-list --limit 10

+-------------------+---------------------+
| name              | dimensions          |
+-------------------+---------------------+
| cpu.frequency_mhz | hostname: devstack  |
|                   | service: monitoring |
| cpu.idle_perc     | hostname: devstack  |
|                   | service: monitoring |
| cpu.idle_time     | hostname: devstack  |
|                   | service: monitoring |
| cpu.percent       | hostname: devstack  |
|                   | service: monitoring |
| cpu.stolen_perc   | hostname: devstack  |
|                   | service: monitoring |
| cpu.system_perc   | hostname: devstack  |
|                   | service: monitoring |
| cpu.system_time   | hostname: devstack  |
|                   | service: monitoring |
| cpu.user_perc     | hostname: devstack  |
|                   | service: monitoring |
| cpu.user_time     | hostname: devstack  |
|                   | service: monitoring |
| cpu.wait_perc     | hostname: devstack  |
|                   | service: m

# Exercise 1: List metrics and filter on name, dimensions and starttime
---
Extend the following command to filter the list and find the previously generated metric.

In [11]:
!monasca metric-list # put your options here

+--------------------------+------------------+
| name                     | dimensions       |
+--------------------------+------------------+
| openstack.handson_status | city: Boston     |
|                          | region: useast   |
|                          | session: monasca |
|                          | state: MA        |
+--------------------------+------------------+


# Example: Create a function to get metrics using the Monasca client
---
Now let's try to create a function which returns the list of metrics using the Monasca Python client.
We will use the previously instantiated 'monasca_client' object and its 'metrics.list' method.
All the CLI options can be used as named arguments.


In [10]:
def get_metrics(names=[None], dimensions={}, limit=10):
    metrics = []
    for name in names:        
        # Invoke the Monasca client
        metrics = metrics + monasca_client.metrics.list(name=name, dimensions=dimensions, limit=limit)
    return metrics

metrics = get_metrics(['openstack.handson_status'])
print json.dumps(metrics, indent=4)

[
    {
        "dimensions": {
            "city": "Boston", 
            "region": "useast", 
            "session": "monasca", 
            "state": "MA"
        }, 
        "name": "openstack.handson_status", 
        "id": "0"
    }, 
    {
        "dimensions": {
            "city": "austin", 
            "region": "uscentral", 
            "session": "monasca", 
            "state": "texas"
        }, 
        "name": "openstack.handson_status", 
        "id": "1"
    }
]


# Metric names
* List all the metric names stored in the service

* GET /v2.0/metrics/names

* Query parameters: tenant_id, offset, limit and dimensions

* Very useful for user-interfaces that need to display the names of all the metrics in the system.

    * Querying all metrics and determining all the distinct metric names is too costly.

# Help: List metric names

In [19]:
!monasca help metric-name-list

usage: monasca metric-name-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>]
                                [--tenant-id <TENANT_ID>]

List names of metrics.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.
  --tenant-id <TENANT_ID>
                        Retrieve data for t

# Example: List metric names

In [20]:
!monasca metric-name-list --limit 20

+----------------------+
| Name                 |
+----------------------+
| cpu.frequency_mhz    |
| cpu.idle_perc        |
| cpu.idle_time        |
| cpu.percent          |
| cpu.stolen_perc      |
| cpu.system_perc      |
| cpu.system_time      |
| cpu.user_perc        |
| cpu.user_time        |
| cpu.wait_perc        |
| cpu.wait_time        |
| disk.inode_used_perc |
| disk.space_used_perc |
| host_alive_status    |
| io.read_kbytes_sec   |
| io.read_req_sec      |
| io.read_time_sec     |
| io.write_kbytes_sec  |
| io.write_req_sec     |
| io.write_time_sec    |
+----------------------+


# Metrics Measurements
---
Operations for returning measurements.

* GET /v2.0/metrics/measurements

* Name and dimensions supplied as query parameters

* Startime, endtime, offset, limit and group_by supplied.

* Note:

    * If group_by is not specified, metrics must be fully qualified with name and dimensions.

# Help: List Measurements
---

In [21]:
!monasca help measurement-list

usage: monasca measurement-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--endtime <UTC_END_TIME>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>] [--merge_metrics]
                                [--group_by <KEY1,KEY2,...>]
                                [--tenant-id <TENANT_ID>]
                                <METRIC_NAME> <UTC_START_TIME>

List measurements for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to list measurements.
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        paramet

# Excercise 2: List measurement
---
Get the list of 5 last measurements with the metric name 'monasca.thread_count' for service 'monitoring' within the last 5 minutes.

In [23]:
!monasca measurement-list --dimensions 'service=monitoring' monasca.thread_count -5   # put your command here

+----------------------+--------------------------+--------------------------+--------------+------------+
| name                 | dimensions               | timestamp                | value        | value_meta |
+----------------------+--------------------------+--------------------------+--------------+------------+
| monasca.thread_count | hostname: devstack       | 2017-05-01T10:24:54.553Z |        9.000 |            |
|                      | component: monasca-agent | 2017-05-01T10:25:09.555Z |        9.000 |            |
|                      | service: monitoring      | 2017-05-01T10:25:24.553Z |        9.000 |            |
|                      |                          | 2017-05-01T10:25:39.555Z |        9.000 |            |
|                      |                          | 2017-05-01T10:25:54.555Z |        9.000 |            |
|                      |                          | 2017-05-01T10:26:09.562Z |        9.000 |            |
|                      |             

# Exercise 3: List measurements by dimension
---
Get the list of measurements with the metric name 'disk.space_used_perc' within last 2 minutes. Please compare with the metric-list and try out using 'group_by' option.


In [28]:
!monasca  # put your command here

+----------------------+---------------------------------------+--------------------------+--------------+------------+
| name                 | dimensions                            | timestamp                | value        | value_meta |
+----------------------+---------------------------------------+--------------------------+--------------+------------+
| disk.space_used_perc | mount_point: /                        | 2017-05-01T10:32:24.556Z |       37.100 |            |
|                      |                                       | 2017-05-01T10:32:39.557Z |       37.100 |            |
|                      |                                       | 2017-05-01T10:32:54.558Z |       37.100 |            |
|                      |                                       | 2017-05-01T10:33:09.560Z |       37.100 |            |
|                      |                                       | 2017-05-01T10:33:24.557Z |       37.100 |            |
|                      |                

# Create a function to get measurements using the Monasca Client
---
Let's try to use client's method 'metrics.list_measurements' to get the list of measurements.
Please use the output of the get_metrics() function defined in the previous example as the first argument of the new function. Remember that the metric is defined by its name and the unique set of dimensions (these are attributes of the metric).

In [33]:
def get_measurements(metrics, start_time = None, end_time = None, limit=100):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:        
        # Invoke the Monasca client
        # please put your code here
        measurements += monasca_client.metrics.list_measurements(
                name=metric['name'],
                dimensions=metric['dimensions'],
                start_time=start_time,
                end_time=end_time,
                limit=limit)
        
    return measurements

# TEST
metrics = get_metrics(['cpu.user_perc','cpu.system_perc'])
measurements = get_measurements(metrics, limit=2)
assert measurements[0]['name'] == 'cpu.user_perc'
assert len(measurements[0]['measurements']) == 2
assert measurements[1]['name'] == 'cpu.system_perc'

print json.dumps(measurements, indent=4)

[
    {
        "name": "cpu.user_perc", 
        "measurements": [
            [
                "2017-05-01T09:37:09.399Z", 
                4.2, 
                {}
            ], 
            [
                "2017-05-01T09:37:24.401Z", 
                4.4, 
                {}
            ]
        ], 
        "id": "0", 
        "columns": [
            "timestamp", 
            "value", 
            "value_meta"
        ], 
        "dimensions": {
            "hostname": "devstack", 
            "service": "monitoring"
        }
    }, 
    {
        "name": "cpu.system_perc", 
        "measurements": [
            [
                "2017-05-01T09:37:09.399Z", 
                3.5, 
                {}
            ], 
            [
                "2017-05-01T09:37:24.401Z", 
                2, 
                {}
            ]
        ], 
        "id": "0", 
        "columns": [
            "timestamp", 
            "value", 
            "value_meta"
        ], 
        "dimens

# Notification Methods
---
Specify the name, type and address to send a notification to.

* GET, POST, PUT, DELETE, PATCH /v2.0/notification-methods

* Notification methods are associated with actions in alarms and are invoked when alarm state transitions occur.

* Supported notification methods are:

  * Email
  
  * PagerDuty
  
  * Webhooks
  
* Plugins available for:

  * Jira
  
  * HipChat
  
  * Slack

# Delete all existing notifications
Let's first delete all existing notifications to prepare the service for creating new ones.

In [11]:
notifications = monasca_client.notifications.list()

for notification in notifications:
    monasca_client.notifications.delete(
        notification_id = notification['id']
    )

# Help: Create Notification Method
---

In [12]:
!monasca help notification-create

usage: monasca notification-create [--period <PERIOD>]
                                   <NOTIFICATION_NAME> <TYPE> <ADDRESS>

Create notification.

Positional arguments:
  <NOTIFICATION_NAME>  Name of the notification to create.
  <TYPE>               The notification type. Type must be EMAIL, WEBHOOK, or
                       PAGERDUTY.
  <ADDRESS>            A valid EMAIL Address, URL, or SERVICE KEY.

Optional arguments:
  --period <PERIOD>    A period for the notification method. Can only be non
                       zero with webhooks


# Example: Create an email notification method

In [13]:
!monasca notification-create "Email Notification" EMAIL john.doe@domain.com

{
  "name": "Email Notification", 
  "links": [
    {
      "href": "http://192.168.10.6:8070/v2.0/notification-methods/5478409f-d348-41dc-930b-fa6986fec0c7", 
      "rel": "self"
    }
  ], 
  "period": 0, 
  "address": "john.doe@domain.com", 
  "type": "EMAIL", 
  "id": "5478409f-d348-41dc-930b-fa6986fec0c7"
}


# Help: List Notification Methods
---

In [27]:
!monasca help notification-list

usage: monasca notification-list [--sort-by <SORT BY FIELDS>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>]

List notifications for this tenant.

Optional arguments:
  --sort-by <SORT BY FIELDS>
                        Fields to sort by as a comma separated list. Valid
                        values are id, name, type, address, created_at,
                        updated_at. Fields may be followed by "asc" or "desc",
                        ex "address desc", to set the direction of sorting.
  --offset <OFFSET LOCATION>
                        The offset used to paginate the return data.
  --limit <RETURN LIMIT>
                        The amount of data to be returned up to the API
                        maximum limit.


# Example: List Notification Methods

In [14]:
!monasca notification-list

+--------------------+--------------------------------------+-------+---------------------+--------+
| name               | id                                   | type  | address             | period |
+--------------------+--------------------------------------+-------+---------------------+--------+
| Email Notification | 5478409f-d348-41dc-930b-fa6986fec0c7 | EMAIL | john.doe@domain.com | 0      |
+--------------------+--------------------------------------+-------+---------------------+--------+


# Alarm Definitions
---
Operations for creating, reading updating and deleting alarm definitions.

* GET, POST /v2.0/alarm-definitions

* GET, PUT, PATCH, DELETE /v2.0/alarm-definitions/{alarm-definition-id}

* Alarm definitions are templates that are used to automatically create alarms based on matching metric names and dimensions

    * The `match-by` paramater is used to match/group metrics together by dimension
    
    * e.g. `--match-by hostname` will create an alarm per unique hostname.

* One alarm definition can result in many alarms

* Simple grammar for creating compound alarm expressions:

   avg(cpu.user_perc{}) > 85 or avg(disk.read_ops{device=vda}, 120) > 1000


* Alarm state (ALARM, OK and UNDETERMINED)

* Actions (notification methods) associated with alarms for state transitions

* User assigned severity (LOW, MEDIUM, HIGH, CRITICAL)



# Delete existing alarm definitions
Let's first delete all existing alarm definitions to prepare the service for creating new ones.

In [29]:
alarm_definitions = monasca_client.alarm_definitions.list()

for definition in alarm_definitions:
    monasca_client.alarm_definitions.delete(alarm_id=definition['id'])    

# Help: Create alarm definition

In [ ]:
!monasca help alarm-definition-create

# Example: Create alarm definition
Create an alarm definition that triggers if the avg of CPU user percent is greater than 80% over 3 60 s periods.

In [ ]:
!monasca alarm-definition-create simple-alarm "max(cpu.user_perc{}, 60) > 80 times 3"

# Example: Create a compound alarm definition
An alarm definition that triggers if either the CPU user or system percent is greater than 80% over 3 60 second periods.

In [ ]:
!monasca alarm-definition-create compound-alarm "avg(cpu.user_perc{}, 60) > 80 times 3 or avg(cpu.system_perc{}, 60) > 80 times 3"

# Alarm definition match_by

* As metrics are consumed by the Threshold Engine they are filtered and checked if they match the definitions that have been specified, based on the metric name and dimensions.

* If a match is found and it is the first one, a new alarm is created.

* If the match_by parameter is specified, metrics are grouped into alarms that match the specified dimensions.

# Example: Two alarm definitions
1. One with no match_by parameter
2. One with match_by set on hostname

In [ ]:
!monasca alarm-definition-create "match name only" "max(test-metric{}) > 80"
!monasca alarm-definition-create --match-by hostname "match by hostname" "max(test-metric{}) > 80"

# Example: Send two metrics
1. One with a name of test-metric and a hostname set to foo
2. One with a name of test-metric and a hostname set to bar

In [ ]:
!monasca metric-create --dimensions hostname=foo test-metric 0.0
!monasca metric-create --dimensions hostname=bar test-metric 0.0

# Quiz: How many alarms were created?

In [ ]:
!monasca alarm-list --metric-name "test-metric"

Three alarms were created:

1. There is one alarm with name "match name only". It has two metrics that are associated with it.

2. There are two alarms with name "match by hostname". Each alarm has one metric that is associated with it, based on the hostnames of `foo` and `bar`.

# UNDETERMINED alarm state
* There are three states of an alarm, `OK`, `ALARM` and `UNDETERMINED`.

* The `UNDETERMINED` alarm state occurs when metrics are no longer being received by an alarm.

* For example, a service is disabled, a system goes down, the network experiences a disruption.

* You can define the alarm as `deterministic`. The only allowed states are then: `OK` and `ALARM`.

# Example: Create an alarm definition with an alarm action.

In [ ]:
!monasca alarm-definition-show 07e06d69-2067-486a-aaa2-ae163468f44b

In [ ]:
# Get the ID of the pagerduty notification method
notifications = monasca_client.notifications.list(sort_by='TYPE')
pagerduty_notification_id = None
for notification in notifications:
    if notification['type'] == 'PAGERDUTY':
        pagerduty_notification = notification   
        pagerduty_notification_id = pagerduty_notification['id']

# Create the alarm        
monasca_client.alarm_definitions.create(name='bootcamp-test',
                                        expression='max(cpu.user_perc{}) > 0',
                                        alarm_actions=pagerduty_notification_id)

# Alarms
---
Alarms are created when incoming metrics match alarm definitions


* GET /v2.0/alarms
* GET, PUT, PATCH, DELETE /v2.0/alarms/{alarm-id}

* Query Parameters

    * alarm_definition_id (string, optional) - Alarm definition ID to filter by.

    * metric_name (string(255), optional) - Name of metric to filter by.

    * metric_dimensions ({string(255): string(255)}, optional) - Dimensions of metrics to filter by specified as a comma separated array of (key, value) pairs as `key1:value1,key1:value1, ...`

    * state (string, optional) - State of alarm to filter by, either `OK`, `ALARM` or `UNDETERMINED`.

    * state_updated_start_time (string, optional) - The start time in ISO 8601 combined date and time format in UTC.
    
    * sort-by: Fields to sort by
    
    * offset, limit


# Help: Alarms list

In [ ]:
!monasca help alarm-list

# Example: Alarm list

In [ ]:
!monasca alarm-list --sort-by severity

# Alarm Counts
---
Provides the ability to query counts of alarms using a number of filter and group-by query parameters.

* GET /v2.0/alarms/counts

* Primarily used in summary/overview dashboards to show the number of alarms in the OK, ALARM and UNDETERMINED state.

* Queries are processed in-database, not client-side.

* A number of filter and group-by fields are supported.

# Help: Alarm counts

In [ ]:
!monasca help alarm-count

# Example: Alarm counts

In [ ]:
!monasca alarm-count --metric-dimensions service=monitoring --group-by state,dimension_name

# Agent
---
* A Python monitoring agent
* Push model
* Agent is installed on the systems that we want to monitor
* Collects metrics by running a set of collection plugins every X amount of seconds 
* Collection plugins are enabled by detection plugins
* Detection plugins generate yaml config files that the collection plugins read from
* The agent has a monasca-setup command line tool that helps configure the agent and run detection plugins

# Agent Metrics and Plugins

* System metrics (cpu, memory, network, filesystem, …)

* Service metrics

* RabbitMQ, MySQL, Kafka, and many others

* Application metrics

    * Built-in statsd daemon

    * Python monasca-statsd library: Adds support for dimensions

* VM system metrics

* Active checks

    * HTTP status checks and response times
    
    * System up/down checks (ping and ssh)
    
* Support for Nagios and checkmk

* Extensible/Pluggable: Additional services can be easily added


# Monasca-setup options

In [ ]:
!/opt/monasca-agent/bin/monasca-setup -h

# Monasca-setup configuring agent

In [17]:
!sudo cat /usr/local/bin/monasca-reconfigure
#monasca_agent_setup_command = shell.run(["sudo", "cat", "/usr/local/bin/monasca-reconfigure"])
#print monasca_agent_setup_command.output

#!/bin/sh
'/opt/monasca-agent/bin/monasca-setup' \
    -u 'monasca-agent' \
    -p 'password' \
     -s 'monitoring'  \
    --keystone_url 'http://192.168.10.6:35357/v3' \
    --project_name 'mini-mon' \
     --monasca_url 'http://192.168.10.6:8070/v2.0'  \
     \
     --check_frequency '15'  \
     \
     --log_level 'WARN'  \
    --overwrite \
    --system_only \
    --monasca_statsd_port 8125


# Monasca agent configuration file

In [18]:
!sudo cat /etc/monasca/agent/agent.yaml

#monasca_agent_setup_command = shell.run(["sudo", "cat", "/etc/monasca/agent/agent.yaml"])
#print monasca_agent_setup_command.output

Api:
  amplifier: 0
  backlog_send_rate: 1000
  ca_file: null
  endpoint_type: null
  insecure: false
  keystone_url: http://192.168.10.6:35357/v3
  max_buffer_size: 1000
  max_measurement_buffer_size: -1
  password: password
  project_domain_id: null
  project_domain_name: null
  project_id: null
  project_name: mini-mon
  region_name: null
  service_type: null
  url: http://192.168.10.6:8070/v2.0
  user_domain_id: null
  user_domain_name: null
  username: monasca-agent
Logging:
  collector_log_file: /var/log/monasca/agent/collector.log
  forwarder_log_file: /var/log/monasca/agent/forwarder.log
  log_level: WARN
  statsd_log_file: /var/log/monasca/agent/statsd.log
Main:
  check_freq: 15
  collector_restart_interval: 24
  dimensions:
    service: monitoring
  hostname: devstack
  num_collector_threads: 1
  pool_full_max_retries: 4
  sub_collection_warn: 6
Statsd:
  monasca_statsd_port: 8125


# Agent Detection Plugins

* Run after initial configuration is run
* List of avaiable plugins 


# Run Kafka Detection Plugin

In [ ]:
monasca_agent_mysql_detection_run = shell.run(["sudo", "/opt/monasca-agent/bin/monasca-setup", "-d", "kafka"])
print monasca_agent_mysql_detection_run.stderr_output

# Example: Query the Kafka consumer lag metrics

In [ ]:
!monasca metric-list --name kafka.consumer_lag

# Example: Query the Kafka consumer lag measurements

In [ ]:
!monasca measurement-list kafka.consumer_lag --dimensions consumer_group=1_metrics -10 --limit 10

# Example: Display Kafka consumer lag using Plotly

In [ ]:
metrics = get_metrics(names=['kafka.consumer_lag'])
statistics = get_statistics(metrics, interval=600)
df = df_from_statistics(statistics, group_by=['consumer_group'])
df.iplot(kind='line', fill=True, xTitle='Date', yTitle='Value', title='Statistics')

# Example: Display a boxplot of Kafka consumer lag using Plotly

In [ ]:
df.iplot(kind='box')

# Example: Detection yaml configuration

In [ ]:
monasca_agent_detection_conf_example = shell.run(["sudo", "cat", "/etc/monasca/agent/conf.d/kafka_consumer.yaml"])
print monasca_agent_detection_conf_example.output

# Developing and Testing
---

* Python and Java codebase

* Monasca DevStack Plugin

* Unit Tests

* Monasca Tempest Tests

* Monasca and OpenStack CI

# Monasca Repos
---
* Monasca is a micro-services message bus based architecture.
* Several repos:
    * monasca-api: The Monasca API, both Python and Java. 
    * monasca-persister: The Monasca Persister, both Python and Java.
    * monasca-thresh: The Monasca Threshold Engine. Java only. Uses Apache Kafka.
    * monasca-notification: The Monasca Notification Engine. Python only.
    * monasca-common: Both Python and Java.
    * monasca-agent: Python.
    * monasca-statsd: Python.
    * monasca-ui: Python.
    * python-monascaclient: Python.
    * puppet-monasca: Puppet.
    * monasca-log-api: Focused on Python.

# Monasca DevStack Plugin
---

* DevStack is the primary developmement environment for OpenStack.

    * See http://docs.openstack.org/developer/devstack/

* The Monasca DevStack plugin installs the Monasca Service, Agent, Horizon Panel, and Grafana

* README at, https://github.com/openstack/monasca-api/tree/master/devstack

* Best way to get started is to deply with Vagrant using the Vagrantfile at, https://github.com/openstack/monasca-api/blob/master/devstack/Vagrantfile.

* The Monasca Log API has a DevStack plugin too.

# Monasca Tempest Tests
---

* [Tempest](http://docs.openstack.org/developer/tempest) is the integration test suite for OpenStack.

* Tempest has an external [Test Plugin Interface](http://docs.openstack.org/developer/tempest/plugin.html) interface that enables anyone to integrate an external test suite.

* There is a [Monasca Tempest Plugin](https://github.com/openstack/monasca-api/tree/master/monasca_tempest_tests)

* Currently, there are around 150 Tempest Tests written for Monasca that run in around 5 minutes.

# Monasca and OpenStack CI
---

* Monasca is fully integrated in the OpenStack CI system.

* Gated jobs are run with each commit on both the Python and Java components through the normal gates and all the Monasca Tempest tests using the Monasca DevStack Plugin.

# Contact the Monasca Team
---
Here are a couple of ways of getting in-touch with us.

* Monasca Weekly Meetings

    * Weekly on Wednesday at 1500 UTC in #openstack-meeting-3 (IRC webclient)
    
* Monasca in IRC

    * #openstack-monasca

# What's new?
---
* Many enhancements for filtering and sorting resources that return arrays. Focused on improving user experience.

* Multiple metrics (in progress): Allow multiple metrics to be returned in a single request. Improves query performance.

* Sporadic metrics (in progress): Supports event based metrics. Metrics that occur infrequently or when events occur. Support events and primarily focused on supporting the Monasca Log Metrics component.

* Periodic notifications (in progress): Send notifications more than once. Improves Heat auto-scaling.

* VM metrics for `host_alive` and `ping_check`. Enables detection of VM status.

* Log API.

# Next Steps
---

* monasca-transform: A transform and aggregation micro-service for Monasca.

    * blueprint: https://blueprints.launchpad.net/monasca/+spec/monasca-transform
    
    * repo: https://github.com/openstack/monasca-transform

* monasca-analytics: An experimental micro-service for Monasca that does anomaly detection and alarm clustering/correlation.

* monasca-events: Complex events processing. Primarily focused on OpenStack notifications.

* Retention periods

* Compression

* More performance improvements

* Dimensions resource.

* Network monitoring: Broadview, Neutron. [OVS Virtual Router Metrics](https://review.openstack.org/#/c/306621/)

Monasca workgroup sessions Wednesday at the Austin Summit Wednesday, 9:00-2:30.

# Horizon and Grafana 2 Demo

[Horizon](http://192.168.10.6)

[Grafana 2.0](http://192.168.10.6:3000/login)

# Example: Add Monasca datasource to Grafana

![Monasca Datasource](monasca-datasource.png)

# Acknowledgments
---

* The OpenStack Technical Committee: Doug Hellman, Flavio Percoco, Thierry Carrez

* Hewlett Packard Enterprise

* Time Warner Cable

* Fujitsu

* Cisco

* NEC

* Cray

* SAP

# Thank you
![Thank you](austin-thank-you.png)

# Resources
---

In [ ]:
!monasca -j measurement-list cpu.user_perc -1

# Example: Alarm with nested alarm subexpression